In [12]:
import json
from fastmcp import Client


In [25]:
# server_url = "http://localhost:8002/intervista" 
server_url = f"https://mcp.tavily.com/mcp/?tavilyApiKey={os.getenv("TAVILY_API_KEY")}" 

# Get tools from MCP Server

In [35]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
mcp_tools = []
async with Client(server_url) as client:
    mcp_tools = await client.list_tools()
    for tool_info in list(mcp_tools):
        print(f"{tool_info}")


name='tavily_search' title=None description='Search the web for real-time information about any topic. Use this tool when you need up-to-date information that might not be available in your training data, or when you need to verify current facts. The search results will include relevant snippets and URLs from web pages. This is particularly useful for questions about current events, technology updates, or any topic that requires recent information.' inputSchema={'properties': {'query': {'description': 'Search query', 'title': 'Query', 'type': 'string'}, 'max_results': {'default': 5, 'description': 'The maximum number of search results to return', 'title': 'Max Results', 'type': 'integer'}, 'search_depth': {'default': 'basic', 'description': "The depth of the search. It can be 'basic' or 'advanced'", 'enum': ['basic', 'advanced'], 'title': 'Search Depth', 'type': 'string'}, 'topic': {'default': 'general', 'description': 'The category of the search. This will determine which of our agent

# Execute Tool Call

In [9]:
async with Client(server_url) as client:

    params = {"customer": "Schutzgarant"}
    result = await client.call_tool_mcp("get_products", params)
    pretty = json.loads(result.content[0].text)
    print(json.dumps(pretty, indent=2, ensure_ascii=False))

{
  "64": {
    "productId": 46,
    "productVariantId": 64,
    "displayName": "FirmenSchutzbrief by ELEMENT",
    "tag": [
      "Versicherung",
      "Geschäftskundentarif",
      "FirmenSchutz",
      "Einzelversicherung",
      "Multigerätetarif"
    ],
    "attribute": [
      {
        "displayName": "DocumentChecker: Id Dokumentenattribut",
        "displayDescription": "",
        "value": "-300",
        "visible": false
      },
      {
        "displayName": "DocumentChecker: Name Attribut an Vertrag",
        "displayDescription": "",
        "value": "Versicherungsschein-Nr.",
        "visible": false
      },
      {
        "displayName": "Mindestvertragslaufzeit",
        "displayDescription": "",
        "value": "12",
        "visible": false
      },
      {
        "displayName": "max. Laufzeit Monate",
        "displayDescription": "",
        "value": "60",
        "visible": false
      },
      {
        "displayName": "Zahlungsart",
        "displayDescription

In [36]:
from openai import OpenAI


def parse_json(data):
    model = os.getenv("STACKIT_MODEL_ID")
    key = os.getenv("STACKIT_KEY")
    base = os.getenv("STACKIT_BASE")

    j_object = None
    if not data or len(data) < 5:
        return None

    js = data
    for attempt in range(3):
        # Find JSON braces
        s = js.find("{")
        e = js.rfind("}") + 1
        if s == -1 or e <= s:
            print(f"Could not find valid JSON braces in: {js}")
            js = data  # reset for LLM
        else:
            try:
                j_object = json.loads(js[s:e])
                return j_object
            except json.JSONDecodeError:
                print(f"Attempt {attempt+1}: JSON decode failed.")

        # Try LLM repair
        try:
            messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a JSON expert function that accepts malformed and broken JSON strings. "
                        "You reply with the corrected version that requires the least changes and fully retains the content and structure. "
                        "Do not comment or explain, just return the corrected JSON."
                    ),
                },
                {"role": "user", "content": js},
            ]
            client = OpenAI(base_url=base, api_key=key)
            response = client.chat.completions.create(model=model, messages=messages)
            js = response.choices[0].message.content
        except Exception as e:
            print(f"An error occurred with LLM: {str(e)}")
            continue

    print(
        f"Failed to parse/fix JSON after 3 attempts. Original: {data}, Last attempt: {js}"
    )
    return None

In [38]:
test_data = '{"hello":"world"}'
test_data = 'abc{"hello":"world"}'
test_data = 'abc{"hello"::"world"}'
test_data = 'abc{"hello"::"world"'


result = parse_json(test_data)

print(result)

Could not find valid JSON braces in: abc{"hello"::"world"
{'hello': 'world'}
